<a href="https://colab.research.google.com/github/seunghoonss/Deep-Learning_Colab/blob/main/CNN_ANIMAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np #넘피

import torchvision.datasets as datasets #datasets 함수 라이브러리

import torchvision.transforms as transforms #transforms 함수 라이브러리

import torch # 토치 라이브러리

import time # 시간 라이브러리

import torchvision # 토치버전 라이브러리

import matplotlib.pyplot as plt #matplotlib

import torch.nn as nn #NN
import torch.nn.functional as F # funtional 라이브러리
import torch.optim as optim # optim 라이브러리
from torchvision import transforms, datasets # 사전 학습된 VGGNET 불러오기

 

start = time.time() # 시작 시간 저장

from torch.utils.data import Dataset, DataLoader #Dataset을 쓰기 위해 Dataset을 상속받아야 하니 Dataset을 impor해라

                                                 #DataLoader를 이용하려면 data를 보관하고 있는 class인 Dataset 형의 calss가 필요하다.

# Transfer learning
transform = transforms.Compose(
    [transforms.Resize((512, 512))]), # 픽셀 크기 224*224 크기 변환
   


# Animal Faces
trainset = torchvision.datasets.ImageFolder(root = "/content/drive/MyDrive/custom_data/train",
                                            transform = transforms.ToTensor())

testset = torchvision.datasets.ImageFolder(root = "/content/drive/MyDrive/custom_data/test",
                                            transform = transforms.ToTensor())




# MNIST dataset
# mnist_train = datasets.MNIST(root='MNIST_data/', #root에 MNIST_data 저장폴더 객체선언

#                              train=True, #60000개의 train set 받아옴

#                              transform=transforms.ToTensor(), #Tensor 형태로 변환

#                              download=True) #다운로드

 

# mnist_test = datasets.MNIST(root='MNIST_data/', #root에 MNIST_data 저장폴더에 객체선언

#                              train=False, #60000개의 train set 받지않음 

#                              transform=transforms.ToTensor(), #Tensor 형태로 변환

#                              download=True) #다운로드

 

print(len(trainset), len(testset)) #총 데이터 수

 

batch_size = 64 #베치사이즈 64

train_loader = torch.utils.data.DataLoader(dataset=trainset, #dataset에 내가 만든 객체이름 넣어주기

                                           batch_size=batch_size, #베치사이즈 코드

                                           shuffle=True) #섞는 코드
                                          

test_loader = torch.utils.data.DataLoader(dataset=testset, #dataset에 내가 만든 객체이름 넣어주기

                                           batch_size=batch_size, #베치사이즈 코드

                                           shuffle=False) #섞지 않는 코드
                                           




# class DeepNet(torch.nn.Module): #DeepNet라는 이름의 calsss 구현

#   def __init__(self):

#     super(DeepNet, self).__init__() #부모클래스 초기화

 

#     self.l1 = torch.nn.Conv2d(784, 256, kernel_size=5) #입력 784개 256개의 층

#     self.l2 = torch.nn.Conv2d(256, 128, kernel_size=5) #층1의 출력 256개를 받아와 입력 받고 128개 출력

#     self.l3 = torch.nn.Maxpool2d(2)  #층2의 출력 128개를 받아와 입력 받고 64개 출력

#     self.l4 = torch.nn.Linear(64, 10)   #층3의 출력 64개를 받아와 입력 받고 10개 출력

 
#     self.relu = torch.nn.ReLU() #파이토치에 nn패키지 ReLU 함수를 사용

 

#   def forward(self, x): #중간층에서는 relu로 활성화 함수를 이용함

##     x = x.view(-1, 784) #배치사이즈는 알아서, 28*28 데이터 784개의 일렬의 데이터로 변환

 
#     in_size = x.size(0)

#     x = self.conv1(x)

#     x = self.mp(x)

#     x = self.relu(x)


#     x= self.relu(self.mp(self.conv2(x)))

#     x = x.view(in_size, -1)

#     yhat = self.fc(x)

 
#     return yhat


class DeepNet(torch.nn.Module): #DeepNet라는 이름의 calsss 구현

  def __init__(self):

    super(DeepNet, self).__init__() #부모클래스 초기화

 
    self.conv1 = torch.nn.Conv2d(3, 10,  kernel_size=5)

    self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)

    self.mp = torch.nn.MaxPool2d(2)

    self.fc = torch.nn.Linear(312500, 10) #입력 784개 256개의 층


    # self.l2 = torch.nn.Linear(220, 128) #층1의 출력 256개를 받아와 입력 받고 128개 출력

    # self.l3 = torch.nn.Linear(128, 64)  #층2의 출력 128개를 받아와 입력 받고 64개 출력

    # self.l4 = torch.nn.Linear(64, 10)   #층3의 출력 64개를 받아와 입력 받고 10개 출력

 

    self.relu = torch.nn.ReLU() #파이토치에 nn패키지 ReLU 함수를 사용

 

  def forward(self, x): #중간층에서는 relu로 활성화 함수를 이용함

    in_size = x.size(0)

    x = self.conv1(x)
    x = self.mp(x)

    x = self.relu(x)


    x = self.relu(self.mp(self.conv2(x)))

    x = x.view(in_size, -1)

    yhat = self.fc(x)

 
    return yhat

 
device = 'cuda' if torch.cuda.is_available() else 'cpu' #device는 함수가 GPU를 쓸 수 있는 환경이면 True, 아니면 False

 

net = DeepNet().to(device) #GPU에 복사하는 DeepNet 클래스 받는 모델 선언

 

criterion = torch.nn.CrossEntropyLoss() #CrossEntropy 구현하는 객체 criterion

optimizer = torch.optim.Adam(net.parameters(), lr=0.01) #Adam를 구현 하는 객체 optimizer

 

def train(epoch): #train하는 함수 

  net.train()

  for batch_idx, (x, y) in enumerate(train_loader): #train_loader 배치 데이터 받아옴

    x, y = x.to(device), y.to(device) #model에 입력되는 x,y 데이터를 GPU에 복사한다

    optimizer.zero_grad() #객체 초기화

    yhat = net(x) #출력결과 예측하는 코드

    loss = criterion(yhat,y) #loss 계산

    loss.backward() #backward 계산

    optimizer.step() #파라미터 업데이트

    if batch_idx % 10 == 0: #배치 10번째마다 현재 로스값 출력

      print(f'Tran Epoch: {epoch}',

            f'[{batch_idx * len(x)}/{len(train_loader.dataset)} ({batch_idx/len(train_loader)*100:0.2f}%)]', #몇번째 데이터 학습을 하였는지 알 수 있는 코드

            f'\tLoss: {loss.item():0.6f}') #loss 계산값 출력                                                 #얼마만큼 학습을 했는지 알 수 있다

 

def test(): #test하는 함수

  net.eval()

  correct = 0

  for x, y in test_loader: #test_loader 데이터 받아옴

    x, y = x.to(device), y.to(device) #model에 입력되는 x,y 데이터를 GPU에 복사한다

    yhat = net(x) #출력결과 예측하는 코드

    pred = torch.max(yhat,1)[1] #최고값 있는 클래스 index값

    correct += pred.eq(y.data.view_as(pred)).sum() #corret는 에측값 정답값 중 정답을 맞춘 논리 값 개수 더해주는 코드

  print(f'\nTest Accuracy: {correct}/{len(test_loader.dataset)} ({correct/len(test_loader.dataset)*100:0.2f}%)\n') #맞춘개수 확률을 알 수 있다.       

 

for epoch in range(10): #배치 10번째마다 반복

  train(epoch) 

  test()   #test 출력

 

 

print("time :", time.time() - start) #실행 시간 차이 측정

                                     #현재시작 - 시작시간 = 실행시간



test_index = 200 #100번째 이미지


net.eval()


x = testset.data[test_index].float().view(1,3,512,512).to(device) #100번째 이미지 x 데이터
y = testset.targets[test_index].to(device) #100번째 이미지 정답
x, y = x.to(device), y.to(device)

yhat = net(x)

pred = torch.max(yhat, 1)[1].item() #최고값 있는 클래스 index값

print(f'Label: {y}\n, Predicted: {pred} ({correct/len(test_loader.dataset)*100:0.2f}%)') #에측값 ,정답 확인 할 수 있다.

 

plt.imshow(testset.data[test_index], cmap='Greys') #그림 명령 코드

plt.show() #그림 출력



15020 1482


RuntimeError: ignored

['Pet (cat+dog)', 'wild'] ['Pet', 'wild']
14630 1500


In [ ]:
import matplotlib.pyplot as plt #matplotlib
import torch
import numpy as np #넘피

import torchvision.datasets as datasets #datasets 함수 라이브러리

import torchvision.transforms as transforms #transforms 함수 라이브러리


device = 'cuda' if torch.cuda.is_available() else 'cpu' #device는 함수가 GPU를 쓸 수 있는 환경이면 True, 아니면 False

test_index = 200 #100번째 이미지


net.eval()


x = testset[test_index][0].view(1,3,512,512).to(device) #100번째 이미지 x 데이터
y = testset[test_index][1].to(device) #100번째 이미지 정답
x, y = x.to(device), y.to(device)

yhat = net(x)

pred = torch.max(yhat, 1)[1].item() #최고값 있는 클래스 index값

print(f'Prediction: {pred}, Label: {y}') #에측값 ,정답 확인 할 수 있다.

 

plt.imshow(testset.data[test_index], cmap='Greys') #그림 명령 코드

plt.show() #그림 출력

RuntimeError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/custom_data/

car.zip


In [ ]:
!unzip /content/drive/MyDrive/custom_data/car.zip -d /content/drive/MyDrive/custom_data/

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/drive/MyDrive/custom_data/train/SUV/SUV_1629_09062020_145842.jpg  
  inflating: /content/drive/MyDrive/custom_data/train/SUV/SUV_1630_09062020_145842.jpg  
  inflating: /content/drive/MyDrive/custom_data/train/SUV/SUV_1631_09062020_145842.jpg  
  inflating: /content/drive/MyDrive/custom_data/train/SUV/SUV_1632_09062020_145842.jpg  
  inflating: /content/drive/MyDrive/custom_data/train/SUV/SUV_1633_09062020_145842.jpg  
  inflating: /content/drive/MyDrive/custom_data/train/SUV/SUV_1634_09062020_145842.jpg  
  inflating: /content/drive/MyDrive/custom_data/train/SUV/SUV_1635_09062020_145842.jpg  
  inflating: /content/drive/MyDrive/custom_data/train/SUV/SUV_1636_09062020_145842.jpg  
  inflating: /content/drive/MyDrive/custom_data/train/SUV/SUV_1637_09062020_145842.jpg  
  inflating: /content/drive/MyDrive/custom_data/train/SUV/SUV_1638_09062020_145842.jpg  
  inflating: /content/drive/MyDrive/custom_data/train/SUV/SUV_1639_0906202